In [1]:
import sys
import math
import random
import copy
import numbers

import pandas as pd
import numpy as np

import torch
import torch.nn.functional as F
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as tr
from torchvision.transforms import Normalize
from PIL import Image

In [123]:
class QualityDataset(Dataset):
    def __init__(self, csv_path, transforms=None, mean=None, std=None):
        self.csv_path = csv_path
        df = pd.read_csv(self.csv_path)
        self.im_list = df.image_id
        self.quality = df.quality.values
        self.artifact = df.artifact.values
        self.clarity = df.clarity.values
        self.field_def = df.field_def.values
        self.transforms = transforms
        if mean is not None and std is not None:
            self.normalize = Normalize(mean, std)
        else:
            # self.normalize = scale_to_mu_sigma_to_0_1
            self.normalize = lambda x: x
    def __getitem__(self, index):
        # load image and labels
#         img = Image.open(self.im_list[index])
        img = Image.open('data/28_l1.jpg')
        label_quality = self.quality[index]
        label_artifact = self.artifact[index]
        label_clarity = self.clarity[index]
        label_field_def = self.field_def[index]

        if self.transforms is not None:
            img = self.transforms(img)

        return img, label_quality, label_artifact, label_clarity, label_field_def

    def __len__(self):
        return len(self.im_list)

In [124]:
train_dataset = QualityDataset('train_all_qualities.csv', mean=[0,0,0], std=[1,1,1])

In [126]:
train_dataset[0][2]

2

In [96]:
np.unique(train_dataset.quality, return_counts=True)

(array([0, 1]), array([624, 575]))

In [97]:
np.unique(train_dataset.artifact, return_counts=True)

(array([0, 1, 2, 3, 4, 5]), array([598,  47, 240, 146, 131,  37]))

In [98]:
np.unique(train_dataset.clarity, return_counts=True)

(array([0, 1, 2, 3, 4]), array([ 29,  85, 257, 397, 431]))

In [99]:
np.unique(train_dataset.field_def, return_counts=True)

(array([0, 1, 2, 3, 4]), array([  6,  48, 180, 801, 164]))

In [100]:
resize = tr.Resize([512,512])
tensorizer = tr.ToTensor()

In [101]:
transforms = tr.Compose([resize, tensorizer])
train_dataset.transforms = transforms

In [102]:
train_dataset[0][0].shape

torch.Size([3, 512, 512])

In [103]:
train_loader = DataLoader(dataset=train_dataset, batch_size=8,
                          num_workers=8, pin_memory=True, shuffle=True)

In [104]:
batch = next(iter(train_loader))

In [127]:
len(batch)

5

In [105]:
batch[0].shape, batch[1].shape

(torch.Size([8, 3, 512, 512]), torch.Size([8]))

In [122]:
batch[1]

tensor([1, 1, 0, 1, 1, 1, 0, 1])

In [106]:
from torchvision.models import resnet as resnet_imagenet

In [107]:
model = resnet_imagenet.resnet18(pretrained=False)

In [108]:
n_classes = 2+6+5+5
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, n_classes)

In [116]:
n_classes

18

In [109]:
out = model(batch[0])

In [110]:
out.shape

torch.Size([8, 18])

In [134]:
preds_quality = out[:, :2]
preds_artifact = out[:, 2:8]
preds_clarity = out[:, 8:13]
preds_field_def = out[:, 13:]

In [131]:
labels_quality = batch[1]
labels_artifact = batch[2]
labels_clarity = batch[3]
labels_field_def = batch[4]

In [135]:
print(preds_quality.shape, preds_artifact.shape, preds_clarity.shape, preds_field_def.shape)
print(labels_quality.shape, labels_artifact.shape, labels_clarity.shape, labels_field_def.shape)

torch.Size([8, 2]) torch.Size([8, 6]) torch.Size([8, 5]) torch.Size([8, 5])
torch.Size([8]) torch.Size([8]) torch.Size([8]) torch.Size([8])


In [133]:
loss = torch.nn.CrossEntropyLoss(reduction='mean')

In [136]:
loss(preds_quality,labels_quality)

tensor(1.0111, grad_fn=<NllLossBackward>)

In [137]:
loss(preds_artifact,labels_artifact)

tensor(1.9880, grad_fn=<NllLossBackward>)

In [138]:
loss(preds_clarity,labels_clarity)

tensor(1.6288, grad_fn=<NllLossBackward>)

In [139]:
loss(preds_field_def,labels_field_def)

tensor(1.5235, grad_fn=<NllLossBackward>)

In [141]:
joint_loss = torch.mean(torch.stack([loss(preds_quality,labels_quality),
                         loss(preds_artifact,labels_artifact),
                         loss(preds_clarity,labels_clarity),
                         loss(preds_field_def,labels_field_def)]))

In [142]:
joint_loss

tensor(1.5379, grad_fn=<MeanBackward0>)

In [15]:
import pandas as pd

In [149]:
df_q = pd.read_csv('data/train_quality.csv')
df_a = pd.read_csv('data/train_artifact.csv')
df_c = pd.read_csv('data/train_clarity.csv')
df_f = pd.read_csv('data/train_field_def.csv')

In [150]:
df_q.columns = ['image_id', 'quality']
df_a.columns = ['image_id', 'artifact']
df_c.columns = ['image_id', 'clarity']
df_f.columns = ['image_id', 'field_def']

In [151]:
from functools import reduce
df_all_qualities = reduce(lambda x, y: pd.merge(x, y, on='image_id'), 
                          [df_q, df_a, df_c, df_f])

In [152]:
df_all_qualities.to_csv('data/train_all_qualities.csv', index=False)

In [153]:
# df_all_qualities.to_csv('train_all_qualities.csv')

In [154]:
import pandas as pd

In [155]:
df_q = pd.read_csv('data/val_quality.csv')
df_a = pd.read_csv('data/val_artifact.csv')
df_c = pd.read_csv('data/val_clarity.csv')
df_f = pd.read_csv('data/val_field_def.csv')

In [156]:
df_q.columns = ['image_id', 'quality']
df_a.columns = ['image_id', 'artifact']
df_c.columns = ['image_id', 'clarity']
df_f.columns = ['image_id', 'field_def']

In [157]:
from functools import reduce
df_all_qualities = reduce(lambda x, y: pd.merge(x, y, on='image_id'), 
                          [df_q, df_a, df_c, df_f])

In [158]:
df_all_qualities.to_csv('data/val_all_qualities.csv', index=False)

In [159]:
# df_all_qualities.to_csv('val_all_qualities.csv')